In [1]:
from pathlib import Path
import pandas as pd

from utils import *
from dataloaders import *
from data_preparation import *
from allResNets import *
from MobileNetV3 import *
from engine import *
from efficientNet import *
from data_transforms import *

change_to_disk()
data_dir = Path("new_split_1k_all/")
models_path = Path('best_model_4/')
check_dir(models_path)

In [2]:
image_shape = (1080, 1920)
resize_factor = 0.3
resize = np.multiply(image_shape,resize_factor)
resize = [int(resize[0]), int(resize[1])]

data_transform = create_transform(resize=resize, rotate=10, flip_h = True, color_dev=True, transf_tensor=True, normalize=True, sp_noise=False, gauss_noise=False)

BATCH_SIZE = 16
train_dl, validation_dl, test_dl, train_data, validation_data, test_data, class_names = create_dataloaders(data_dir, data_transform, batch_size = BATCH_SIZE)

In [3]:
NUM_EPOCHS = 100
device = get_device()

resnet18 = ResNet18()
resnet50 = ResNet50()
mobnet_sml = MobileNetSmall()
mobnet_lrg = MobileNetLarge()
effnetb0 = EfficientNetB0()
loss_fn = nn.CrossEntropyLoss()

In [8]:
model_name = "ResNet18"
model_folder_path = models_path / model_name
check_dir(model_folder_path)
model_name_folder_path = model_folder_path / model_name

#Train ResNet18
resnet18.to(device)
nadam_optim = torch.optim.NAdam(params=resnet18.parameters())
train_resnet18_results, train_time_resnet18 = train(resnet18, train_dl, validation_dl, optimizer=nadam_optim, loss_fn=loss_fn, epochs=NUM_EPOCHS, name_save=model_name_folder_path, device=device)

best_model_4\ResNet18 does not exist, creating one...


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.8140 | train_acc: 0.6616 | validation_loss: 1.0316 | validation_acc: 0.5875
Epoch: 2 | train_loss: 0.4875 | train_acc: 0.8063 | validation_loss: 0.7042 | validation_acc: 0.7250
Epoch: 3 | train_loss: 0.3799 | train_acc: 0.8506 | validation_loss: 0.5772 | validation_acc: 0.7975
Epoch: 4 | train_loss: 0.3277 | train_acc: 0.8738 | validation_loss: 1.2322 | validation_acc: 0.6725
Epoch: 5 | train_loss: 0.3102 | train_acc: 0.8744 | validation_loss: 1.0628 | validation_acc: 0.7150
Epoch: 6 | train_loss: 0.2730 | train_acc: 0.8925 | validation_loss: 0.4519 | validation_acc: 0.8425
Epoch: 7 | train_loss: 0.2415 | train_acc: 0.9094 | validation_loss: 0.8387 | validation_acc: 0.7300
Epoch: 8 | train_loss: 0.2158 | train_acc: 0.9209 | validation_loss: 0.7313 | validation_acc: 0.7950
Epoch: 9 | train_loss: 0.2056 | train_acc: 0.9228 | validation_loss: 0.3799 | validation_acc: 0.8850
Epoch: 10 | train_loss: 0.1959 | train_acc: 0.9266 | validation_loss: 0.5690 | validation_a

In [9]:
eval_resnet18_results = eval_model(resnet18, test_dl, loss_fn = loss_fn, accuracy_fn=accuracy_fn, device = device, dummy_input= torch.rand(32,3,224,224).to(device))
resnet18_results = [train_resnet18_results, train_time_resnet18, eval_resnet18_results]
final_save(model_folder_path,model_name, NUM_EPOCHS, resnet18, resnet18_results)

  0%|          | 0/25 [00:02<?, ?it/s]

In [4]:
model_name = "ResNet50"
model_folder_path = models_path / model_name
check_dir(model_folder_path)
model_name_folder_path = model_folder_path / model_name

#Train ResNet50
resnet50.to(device)
nadam_optim = torch.optim.NAdam(params=resnet50.parameters())
train_resnet50_results, train_time_resnet50 = train(resnet50, train_dl, validation_dl, optimizer=nadam_optim, loss_fn=loss_fn, epochs=NUM_EPOCHS, name_save=model_name_folder_path, device=device)

best_model_4\ResNet50 does not exist, creating one...


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.1111 | train_acc: 0.5350 | validation_loss: 1.6358 | validation_acc: 0.4900
Epoch: 2 | train_loss: 0.6572 | train_acc: 0.7262 | validation_loss: 1.8097 | validation_acc: 0.5625
Epoch: 3 | train_loss: 0.5022 | train_acc: 0.7956 | validation_loss: 0.8469 | validation_acc: 0.7000
Epoch: 4 | train_loss: 0.4317 | train_acc: 0.8200 | validation_loss: 1.7211 | validation_acc: 0.5075
Epoch: 5 | train_loss: 0.4041 | train_acc: 0.8394 | validation_loss: 3.6861 | validation_acc: 0.5350
Epoch: 6 | train_loss: 0.3477 | train_acc: 0.8600 | validation_loss: 1.2457 | validation_acc: 0.6450
Epoch: 7 | train_loss: 0.3420 | train_acc: 0.8675 | validation_loss: 0.6311 | validation_acc: 0.7450
Epoch: 8 | train_loss: 0.3210 | train_acc: 0.8725 | validation_loss: 0.8132 | validation_acc: 0.6900
Epoch: 9 | train_loss: 0.2928 | train_acc: 0.8834 | validation_loss: 1.9321 | validation_acc: 0.5500
Epoch: 10 | train_loss: 0.2712 | train_acc: 0.8962 | validation_loss: 0.4414 | validation_a

RuntimeError: Caught RuntimeError in DataLoader worker process 7.
Original Traceback (most recent call last):
  File "c:\Users\joaor\anaconda3\lib\site-packages\torch\utils\data\_utils\worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "c:\Users\joaor\anaconda3\lib\site-packages\torch\utils\data\_utils\fetch.py", line 58, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "c:\Users\joaor\anaconda3\lib\site-packages\torch\utils\data\_utils\fetch.py", line 58, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "c:\Users\joaor\anaconda3\lib\site-packages\torchvision\datasets\folder.py", line 231, in __getitem__
    sample = self.transform(sample)
  File "c:\Users\joaor\anaconda3\lib\site-packages\torchvision\transforms\transforms.py", line 95, in __call__
    img = t(img)
  File "c:\Users\joaor\anaconda3\lib\site-packages\torchvision\transforms\transforms.py", line 135, in __call__
    return F.to_t

In [ ]:
eval_resnet50_results = eval_model(resnet50, test_dl, loss_fn = loss_fn, accuracy_fn=accuracy_fn, device = device, dummy_input= torch.rand(32,3,224,224).to(device))
resnet50_results = [train_resnet50_results, train_time_resnet50, eval_resnet50_results]
final_save(model_folder_path,model_name, NUM_EPOCHS, resnet50, resnet50_results)

In [10]:
model_name = "MobileNetSmall"
model_folder_path = models_path / model_name
check_dir(model_folder_path)
model_name_folder_path = model_folder_path / model_name

#Train MobileNetV3 Small
mobnet_sml.to(device)
nadam_optim = torch.optim.NAdam(params=mobnet_sml.parameters())
train_mobnet_sml_results, train_time_mobnet_sml = train(mobnet_sml, train_dl, validation_dl, optimizer=nadam_optim, loss_fn=loss_fn, epochs=NUM_EPOCHS, name_save=model_name_folder_path, device=device)

best_model_4\MobileNetSmall does not exist, creating one...


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.8811 | train_acc: 0.6253 | validation_loss: 1.0472 | validation_acc: 0.6725
Epoch: 2 | train_loss: 0.4791 | train_acc: 0.8091 | validation_loss: 0.9245 | validation_acc: 0.7050
Epoch: 3 | train_loss: 0.3540 | train_acc: 0.8669 | validation_loss: 0.7536 | validation_acc: 0.7225
Epoch: 4 | train_loss: 0.2781 | train_acc: 0.9047 | validation_loss: 0.5117 | validation_acc: 0.8125
Epoch: 5 | train_loss: 0.2294 | train_acc: 0.9166 | validation_loss: 0.7106 | validation_acc: 0.8100
Epoch: 6 | train_loss: 0.2500 | train_acc: 0.9147 | validation_loss: 0.4382 | validation_acc: 0.8550
Epoch: 7 | train_loss: 0.2040 | train_acc: 0.9313 | validation_loss: 0.3047 | validation_acc: 0.8975
Epoch: 8 | train_loss: 0.1925 | train_acc: 0.9334 | validation_loss: 0.4482 | validation_acc: 0.8475
Epoch: 9 | train_loss: 0.1567 | train_acc: 0.9459 | validation_loss: 0.3671 | validation_acc: 0.8600
Epoch: 10 | train_loss: 0.1652 | train_acc: 0.9434 | validation_loss: 0.4203 | validation_a

In [11]:
eval_mobnet_sml_results = eval_model(mobnet_sml, test_dl, loss_fn = loss_fn, accuracy_fn=accuracy_fn, device = device, dummy_input= torch.rand(32,3,224,224).to(device))
mobnet_sml_results = [train_mobnet_sml_results, train_time_mobnet_sml, eval_mobnet_sml_results]
final_save(model_folder_path,model_name, NUM_EPOCHS, mobnet_sml, mobnet_sml_results)

  0%|          | 0/25 [00:02<?, ?it/s]

In [13]:
model_name = "MobileNetLarge"
model_folder_path = models_path / model_name
check_dir(model_folder_path)
model_name_folder_path = model_folder_path / model_name

#Train MobileNetV3 Large
mobnet_lrg.to(device)
nadam_optim = torch.optim.NAdam(params=mobnet_lrg.parameters())
train_mobnet_lrg_results, train_time_mobnet_lrg = train(mobnet_lrg, train_dl, validation_dl, optimizer=nadam_optim, loss_fn=loss_fn, epochs=NUM_EPOCHS, name_save=model_name_folder_path, device=device)

best_model_4\MobileNetLarge does not exist, creating one...


  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 1.0151 | train_acc: 0.5634 | validation_loss: 0.8064 | validation_acc: 0.6575
Epoch: 2 | train_loss: 0.6592 | train_acc: 0.7369 | validation_loss: 0.7113 | validation_acc: 0.7050
Epoch: 3 | train_loss: 0.4750 | train_acc: 0.8141 | validation_loss: 0.5054 | validation_acc: 0.8300
Epoch: 4 | train_loss: 0.3854 | train_acc: 0.8525 | validation_loss: 0.3543 | validation_acc: 0.8600
Epoch: 5 | train_loss: 0.3256 | train_acc: 0.8728 | validation_loss: 0.9768 | validation_acc: 0.6950
Epoch: 6 | train_loss: 0.2699 | train_acc: 0.9038 | validation_loss: 0.8231 | validation_acc: 0.7825
Epoch: 7 | train_loss: 0.2442 | train_acc: 0.9047 | validation_loss: 0.3073 | validation_acc: 0.9050
Epoch: 8 | train_loss: 0.2740 | train_acc: 0.8956 | validation_loss: 0.6223 | validation_acc: 0.7825
Epoch: 9 | train_loss: 0.2437 | train_acc: 0.9113 | validation_loss: 0.5671 | validation_acc: 0.8275
Epoch: 10 | train_loss: 0.2114 | train_acc: 0.9269 | validation_loss: 0.6893 | validation_a

In [14]:
eval_mobnet_lrg_results = eval_model(mobnet_lrg, test_dl, loss_fn = loss_fn, accuracy_fn=accuracy_fn, device = device, dummy_input= torch.rand(32,3,224,224).to(device))
mobnet_lrg_results = [train_mobnet_lrg_results, train_time_mobnet_lrg, eval_mobnet_lrg_results]
final_save(model_folder_path, model_name, NUM_EPOCHS, mobnet_lrg, mobnet_lrg_results)

  0%|          | 0/25 [00:02<?, ?it/s]

In [5]:
model_name = "EfficientNetB0"
model_folder_path = models_path / model_name
check_dir(model_folder_path)
model_name_folder_path = model_folder_path / model_name

#Train EfficientNet B0
effnetb0.to(device)
nadam_optim = torch.optim.NAdam(params=effnetb0.parameters())
train_effnetb0_results, train_time_effnetb0 = train(effnetb0, train_dl, validation_dl, optimizer=nadam_optim, loss_fn=loss_fn, epochs=NUM_EPOCHS, name_save=model_name_folder_path, device=device)

best_model_4\EfficientNetB0 does not exist, creating one...


  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
eval_effnetb0_results = eval_model(effnetb0, test_dl, loss_fn = loss_fn, accuracy_fn=accuracy_fn, device = device, dummy_input= torch.rand(32,3,224,224).to(device))
effnetb0_results = [train_effnetb0_results, train_time_effnetb0, eval_effnetb0_results]
final_save(model_folder_path, model_name, NUM_EPOCHS, effnetb0, effnetb0_results)

  0%|          | 0/25 [00:02<?, ?it/s]